In [ ]:
import numpy as np

from dataclasses import dataclass
from datasets import load_dataset, load_from_disk
from huggingface_hub import login, list_models
from transformers import DecisionTransformerConfig, Trainer, TrainingArguments

from model.ardt_naive import SingleAgentRobustDT
from model.ardt_full import TwoAgentRobustDT
from model.ardt_utils import DecisionTransformerGymDataCollator

#
import warnings
warnings.filterwarnings('ignore')

from access_tokens import WRITE_TOKEN

In [ ]:
login(token=WRITE_TOKEN)

## Configs

In [ ]:
envs = {
    0: "walker2d-expert-v2",
    1: "halfcheetah-expert-v2",
}

chosen_env = envs[1]

In [ ]:
agent = {
    0: SingleAgentRobustDT,
    1: TwoAgentRobustDT
}

chosen_agent = agent[1]

## Loading and exploring the dataset: halfcheetah (expert)

Some notes:
* This is a multi-dimensional, continuous environment. States are represented by 17 continuous dimensions; actions are represented by 7 continuous dimensions.
* The state space includes the positions and velocities of multiple body parts of the robotic cheetah, which are continuous, unbounded, real-valued quantities.
* The action space consists of torques applied to the joints, which are real-valued and thus continuous. They are however limited to the interval [-1, 1]. 

For more details: https://www.gymlibrary.dev/environments/mujoco/half_cheetah/

In [ ]:
# # from local, pre-saved
# dataset = load_from_disk("./datasets/toy_dataset")

In [ ]:
# from HF default
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", chosen_env)['train']

In [ ]:
# run if we want to add a 0 adversary
pr_actions = dataset['actions']
new_pr_actions = []
adv_actions =[]
for tr in pr_actions:
    pr_l = []
    adv_l = []
    for a in tr:
        adv = np.array(a) * 0
        pr = np.array(a)
        pr_l.append(list(pr))
        adv_l.append(list(adv))
    new_pr_actions.append(pr_l)
    adv_actions.append(adv_l)

dataset = dataset.add_column('pr_actions', new_pr_actions)
dataset = dataset.add_column('adv_actions', adv_actions)
dataset = dataset.remove_columns(['actions'])
dataset.save_to_disk('./datasets/default_dataset')

In [ ]:
# # run if we want to add a noisy random adversary
# pr_actions = dataset['actions']
# new_pr_actions = []
# adv_actions =[]
# for tr in pr_actions:
#     pr_l = []
#     adv_l = []
#     for a in tr:
#         adv = np.array(a) * 0.1 * np.random.choice([-1, 1], size=len(a)) * np.random.rand(len(a))
#         pr = np.array(a) + adv
#         pr_l.append(list(pr))
#         adv_l.append(list(adv))
#     new_pr_actions.append(pr_l)
#     adv_actions.append(adv_l)

# dataset = dataset.add_column('pr_actions', new_pr_actions)
# dataset = dataset.add_column('adv_actions', adv_actions)
# dataset = dataset.remove_columns(['actions'])
# dataset.save_to_disk('./datasets/toy_dataset')

In [ ]:
print("Dataset elements: ", dataset[0].keys())
print("Number of steps: ", len(dataset[0]['observations']))
print("Size of state representation: ", len(dataset[0]['observations'][0]))
print("Size of action representation: ", len(dataset[0]['pr_actions'][0]))
print("Size of action representation: ", len(dataset[0]['adv_actions'][0]))
print("Reward type: ", type(dataset[0]['rewards'][0]))
print("Done flag: ", type(dataset[0]['dones'][0]))

## Setting up the model

In [ ]:
RETURNS_SCALE = 1000.0
CONTEXT_SIZE = 20

In [ ]:
collator = DecisionTransformerGymDataCollator(dataset, context_size=CONTEXT_SIZE, returns_scale=RETURNS_SCALE)
config = DecisionTransformerConfig(state_dim=collator.state_dim, 
                                   pr_act_dim=collator.pr_act_dim,
                                   adv_act_dim=collator.adv_act_dim,
                                   max_ep_len=collator.max_ep_len,
                                   context_size=collator.context_size,
                                   state_mean=list(collator.state_mean),
                                   state_std=list(collator.state_std),
                                   scale=collator.scale,
                                   lambda1=0.3,
                                   lambda2=10.0,
                                   max_return=1000) # FIXME completely random, potentially not needed
model = chosen_agent(config)

## Training the model

In [ ]:
my_env_name = "ardt-" + chosen_env.split("-")[0]
models = sorted([m.modelId.split("/")[-1] for m in list_models(author="afonsosamarques")])
models = [m for m in models if my_env_name in m]
if len(models) > 0:
    latest_version = [m.split("-")[-1][1:] for m in models][-1]
    new_version = "v" + str(int(latest_version) + 1)
else:
    new_version = "v0"
model_name = my_env_name + "-" + new_version
print(model_name)

In [ ]:
# we use the same hyperparameters as in the authors original implementation, but train for fewer iterations
training_args = TrainingArguments(
    output_dir="./agents/" + model_name,
    remove_unused_columns=False,
    num_train_epochs=300,
    per_device_train_batch_size=64,
    learning_rate=1e-4,
    weight_decay=1e-4,
    warmup_ratio=0.1,
    optim="adamw_torch",
    max_grad_norm=0.25,
    use_mps_device=False,
    report_to="none",
    push_to_hub=True,
    hub_model_id=model_name,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=collator,
)

trainer.train()
trainer.save_model()
# trainer.push_to_hub()